# COW vs MOR 직접 비교 실험

동일한 데이터와 동일한 작업을 COW/MOR 테이블에서 각각 수행하고,  
**실행 시간, 파일 구조, 메타데이터 계층**을 정량적으로 비교합니다.

COW와 MOR은 **쓰기 방식만 다를 뿐, 동일한 Iceberg 테이블 구조**를 사용합니다.  
이 실험을 통해 두 전략이 **파일 시스템 수준에서 어떻게 다르게 동작하는지** 확인합니다.

## 실험 설계

| 단계 | 작업 | 데이터 규모 | 설명 |
|------|------|------------|------|
| 0 | Warmup | — | JVM/Iceberg 초기화 비용 제거 |
| 1 | INSERT | 2,000건 | 초기 데이터 적재 (COW/MOR 동일) |
| 2 | UPDATE | 600건 (30%) | 상태 변경 → COW는 파일 재작성, MOR는 Delete File + 새 데이터 |
| 3 | DELETE | 200건 (10%) | 삭제 → COW는 파일 재작성, MOR는 Delete File만 추가 |
| 4 | SELECT | 전체 | 읽기 → COW는 데이터만, MOR는 Delete File 병합 필요 |

동일한 시나리오를 COW와 MOR 양쪽에서 실행하여 차이를 관찰합니다.

> **Warmup**: Spark/JVM은 첫 작업에서 클래스 로딩, JIT 컴파일 등의 초기화 비용이 발생합니다.  
> 먼저 실행되는 쪽이 불리해지는 것을 방지하기 위해, 별도의 테이블에 dummy 작업을 수행합니다.

## 환경 설정

In [1]:
import sys
sys.path.append('..')

import time
import pandas as pd

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree, count_files, total_size, show_metadata_hierarchy

In [2]:
spark = create_spark_session()

COW_TABLE = "demo.lab.cmp_cow_orders"
MOR_TABLE = "demo.lab.cmp_mor_orders"
COW_PATH = "/home/jovyan/data/warehouse/lab/cmp_cow_orders"
MOR_PATH = "/home/jovyan/data/warehouse/lab/cmp_mor_orders"

Spark + Iceberg 세션 준비 완료 (warehouse: file:///home/jovyan/data/warehouse)


## 테이블 생성

동일한 스키마, 동일한 파티셔닝 — **TBLPROPERTIES만 다릅니다**.

In [3]:
# 기존 테이블 정리
spark.sql(f"DROP TABLE IF EXISTS {COW_TABLE}")
spark.sql(f"DROP TABLE IF EXISTS {MOR_TABLE}")

# COW 테이블
spark.sql(f"""
CREATE TABLE {COW_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
TBLPROPERTIES (
    'write.delete.mode'='copy-on-write',
    'write.update.mode'='copy-on-write',
    'write.merge.mode'='copy-on-write'
)
""")
print(f"COW 테이블 생성 완료: {COW_TABLE}")

# MOR 테이블
spark.sql(f"""
CREATE TABLE {MOR_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
TBLPROPERTIES (
    'write.delete.mode'='merge-on-read',
    'write.update.mode'='merge-on-read',
    'write.merge.mode'='merge-on-read'
)
""")
print(f"MOR 테이블 생성 완료: {MOR_TABLE}")

# ── JVM Warmup ──
# 별도 dummy 테이블에 INSERT → UPDATE → DELETE → DROP 수행하여
# Spark/JVM/Iceberg 초기화 비용을 미리 소모합니다.
WARMUP_TABLE = "demo.lab.cmp_warmup"
spark.sql(f"DROP TABLE IF EXISTS {WARMUP_TABLE}")
spark.sql(f"""
CREATE TABLE {WARMUP_TABLE} (id BIGINT, val STRING)
USING ICEBERG
""")
warmup_df = spark.createDataFrame([(i, "x") for i in range(100)], ["id", "val"])
warmup_df.writeTo(WARMUP_TABLE).append()
spark.sql(f"UPDATE {WARMUP_TABLE} SET val = 'y' WHERE id < 10")
spark.sql(f"DELETE FROM {WARMUP_TABLE} WHERE id >= 90")
spark.sql(f"SELECT COUNT(*) FROM {WARMUP_TABLE}").collect()
spark.sql(f"DROP TABLE {WARMUP_TABLE}")
print("JVM Warmup 완료 — 이후 측정은 초기화 비용이 제거된 상태")

COW 테이블 생성 완료: demo.lab.cmp_cow_orders
MOR 테이블 생성 완료: demo.lab.cmp_mor_orders
JVM Warmup 완료 — 이후 측정은 초기화 비용이 제거된 상태


---
## 단계 1: INSERT 2,000건

동일한 데이터(seed=42)를 양쪽 테이블에 삽입합니다.  
INSERT는 COW/MOR 모두 **새 데이터 파일을 생성**하는 동일한 동작이므로, 시간 차이가 거의 없어야 합니다.

In [4]:
# 동일한 데이터 생성 (2,000건)
orders = generate_orders(num_records=2000, seed=42)
df = to_spark_df(spark, orders)
df.cache()
df.count()  # 캐시 실체화

# COW INSERT
start = time.time()
df.writeTo(COW_TABLE).append()
cow_insert_time = time.time() - start
print(f"COW INSERT 시간: {cow_insert_time:.3f}초")

# MOR INSERT
start = time.time()
df.writeTo(MOR_TABLE).append()
mor_insert_time = time.time() - start
print(f"MOR INSERT 시간: {mor_insert_time:.3f}초")

df.unpersist()

cow_count_after_insert = spark.sql(f"SELECT COUNT(*) FROM {COW_TABLE}").collect()[0][0]
print(f"\n양쪽 테이블 모두 {cow_count_after_insert}건 삽입 완료")
print(f"INSERT 시간 차이: {abs(cow_insert_time - mor_insert_time):.3f}초 (거의 동일해야 정상)")

COW INSERT 시간: 0.494초
MOR INSERT 시간: 0.312초

양쪽 테이블 모두 2000건 삽입 완료
INSERT 시간 차이: 0.182초 (거의 동일해야 정상)


### 관찰 포인트 — INSERT

INSERT는 COW/MOR 차이가 없습니다. 둘 다 **새 데이터 파일을 생성**할 뿐이므로, Warmup 이후에는 시간이 거의 동일합니다.

- COW: 새 Parquet 파일 생성
- MOR: 새 Parquet 파일 생성
- **차이점 없음** — 기존 데이터를 수정하지 않으므로 전략이 개입하지 않음

---
## 단계 2: UPDATE 600건 (전체의 30%)

order_id 기준으로 처음 600건의 상태를 변경합니다.  
**여기서부터 COW와 MOR의 차이가 드러납니다.**

- **COW**: 변경된 행이 속한 파티션의 데이터 파일을 **통째로 재작성** (영향받지 않은 행도 다시 씀)
- **MOR**: 기존 행을 가리키는 **Positional Delete File** + 변경된 값의 **새 데이터 파일**만 추가

In [5]:
# COW UPDATE
start = time.time()
spark.sql(f"UPDATE {COW_TABLE} SET status = 'refunded' WHERE order_id <= 600")
cow_update_time = time.time() - start
print(f"COW UPDATE 시간: {cow_update_time:.3f}초")

# MOR UPDATE
start = time.time()
spark.sql(f"UPDATE {MOR_TABLE} SET status = 'refunded' WHERE order_id <= 600")
mor_update_time = time.time() - start
print(f"MOR UPDATE 시간: {mor_update_time:.3f}초")

ratio = cow_update_time / mor_update_time if mor_update_time > 0 else float('inf')
print(f"\nCOW/MOR 비율: {ratio:.2f}x — COW가 {ratio:.1f}배 느림")

COW UPDATE 시간: 0.537초
MOR UPDATE 시간: 0.634초

COW/MOR 비율: 0.85x — COW가 0.8배 느림


### 관찰 포인트 — UPDATE

| | COW | MOR |
|---|---|---|
| **동작** | 영향받은 파티션의 데이터 파일을 통째로 재작성 | Delete File + 새 데이터 파일만 추가 |
| **I/O** | 변경 안 된 행도 다시 써야 함 → **쓰기 증폭** | 변경된 행만큼만 기록 → 최소 I/O |
| **결과 파일** | 기존 파일 DELETED → 새 파일 ADDED | 기존 파일 유지 + Delete File ADDED + 새 데이터 ADDED |

> 600건(30%) UPDATE에서 COW가 느린 이유:  
> 3개 파티션 모두 영향을 받으므로 **모든 파티션의 데이터 파일을 재작성**해야 합니다.

---
## 단계 3: DELETE 200건 (전체의 10%)

order_id 601~800번 주문을 삭제합니다.

- **COW**: 해당 행이 빠진 **새 데이터 파일을 재작성**
- **MOR**: 삭제 위치만 기록한 **Positional Delete File**만 추가

In [6]:
# COW DELETE
start = time.time()
spark.sql(f"DELETE FROM {COW_TABLE} WHERE order_id BETWEEN 601 AND 800")
cow_delete_time = time.time() - start
print(f"COW DELETE 시간: {cow_delete_time:.3f}초")

# MOR DELETE
start = time.time()
spark.sql(f"DELETE FROM {MOR_TABLE} WHERE order_id BETWEEN 601 AND 800")
mor_delete_time = time.time() - start
print(f"MOR DELETE 시간: {mor_delete_time:.3f}초")

ratio = cow_delete_time / mor_delete_time if mor_delete_time > 0 else float('inf')
print(f"\nCOW/MOR 비율: {ratio:.2f}x — COW가 {ratio:.1f}배 느림")

COW DELETE 시간: 0.531초
MOR DELETE 시간: 0.352초

COW/MOR 비율: 1.51x — COW가 1.5배 느림


### 관찰 포인트 — DELETE

| | COW | MOR |
|---|---|---|
| **동작** | 삭제 행이 빠진 새 파일 재작성 | Positional Delete File만 추가 |
| **I/O** | UPDATE와 마찬가지로 **전체 파일 재작성** | 삭제 위치(file_path + pos)만 기록 → **매우 작은 I/O** |
| **결과** | 기존 데이터 파일 DELETED → 새 파일 ADDED | 기존 데이터 파일 유지 + Delete File ADDED |

> DELETE는 UPDATE보다 MOR의 이점이 더 큽니다.  
> UPDATE는 새 값을 기록한 데이터 파일이 필요하지만, DELETE는 **Delete File만으로 충분**합니다.

---
## 단계 4: SELECT — 읽기 성능 비교

전체 데이터를 읽는 시간을 비교합니다.

- **COW**: 데이터 파일만 읽으면 됨 (이미 최신 상태)
- **MOR**: 데이터 파일 + Delete File을 **병합(merge)**해야 정확한 결과를 얻음

In [7]:
# COW SELECT
start = time.time()
cow_count = spark.sql(f"SELECT COUNT(*) FROM {COW_TABLE}").collect()[0][0]
cow_read_time = time.time() - start
print(f"COW SELECT 시간: {cow_read_time:.3f}초 (레코드 수: {cow_count})")

# MOR SELECT
start = time.time()
mor_count = spark.sql(f"SELECT COUNT(*) FROM {MOR_TABLE}").collect()[0][0]
mor_read_time = time.time() - start
print(f"MOR SELECT 시간: {mor_read_time:.3f}초 (레코드 수: {mor_count})")

ratio = mor_read_time / cow_read_time if cow_read_time > 0 else float('inf')
print(f"\nMOR/COW 비율: {ratio:.2f}x")
print(f"레코드 수 일치 여부: {cow_count == mor_count} ({cow_count}건)")

COW SELECT 시간: 0.062초 (레코드 수: 1800)
MOR SELECT 시간: 0.136초 (레코드 수: 1800)

MOR/COW 비율: 2.19x
레코드 수 일치 여부: True (1800건)


### 관찰 포인트 — SELECT

| | COW | MOR |
|---|---|---|
| **읽기 대상** | 데이터 파일만 | 데이터 파일 + Delete File 병합 |
| **Merge 비용** | 없음 | Delete File 기반 필터링 필요 |
| **특성** | 쓰기 시 이미 최적화 완료 | 읽기 시 병합 비용 발생 |

> 소규모 데이터에서는 읽기 차이가 미미하지만, Delete File이 누적될수록 MOR의 읽기 비용이 증가합니다.  
> 이를 해결하기 위해 **정기적인 컴팩션(compaction)**이 필요합니다 (→ 4_optimization 모듈).

---
## 파일 시스템 비교

시간 측정만으로는 **왜** 차이가 나는지 알기 어렵습니다.  
파일 시스템 수준에서 **데이터 파일 수, Delete File 유무, 메타데이터 계층**을 직접 비교합니다.

In [8]:
print("=" * 60)
print("COW 테이블 파일 구조:")
print("=" * 60)
show_tree(COW_PATH)

cow_data_files = count_files(COW_PATH, ext=".parquet")
cow_total = total_size(COW_PATH)

print(f"\nParquet 파일: {cow_data_files}개, 총 크기: {cow_total:,} bytes")

print("\n" + "=" * 60)
print("MOR 테이블 파일 구조:")
print("=" * 60)
show_tree(MOR_PATH)

mor_data_files = count_files(MOR_PATH, ext=".parquet")
mor_total = total_size(MOR_PATH)

print(f"\nParquet 파일: {mor_data_files}개, 총 크기: {mor_total:,} bytes")

COW 테이블 파일 구조:
├── data/
│   ├── order_date_month=2024-01/
│   │   ├── 00000-24-f9838e57-9090-41d5-99f0-a749a587d9ac-00001.parquet  (7.3 KB)
│   │   ├── 00000-35-9ba3c68f-d801-4864-afc5-91950d1755d0-00001.parquet  (7.3 KB)
│   │   └── 00000-40-e3e46849-f884-450a-9d73-1f9ff95f495b-00001.parquet  (6.6 KB)
│   ├── order_date_month=2024-02/
│   │   ├── 00000-24-f9838e57-9090-41d5-99f0-a749a587d9ac-00002.parquet  (7.6 KB)
│   │   ├── 00000-35-9ba3c68f-d801-4864-afc5-91950d1755d0-00002.parquet  (7.5 KB)
│   │   └── 00000-40-e3e46849-f884-450a-9d73-1f9ff95f495b-00002.parquet  (7.0 KB)
│   └── order_date_month=2024-03/
│       ├── 00000-24-f9838e57-9090-41d5-99f0-a749a587d9ac-00003.parquet  (8.4 KB)
│       ├── 00000-35-9ba3c68f-d801-4864-afc5-91950d1755d0-00003.parquet  (8.3 KB)
│       └── 00000-40-e3e46849-f884-450a-9d73-1f9ff95f495b-00003.parquet  (7.9 KB)
└── metadata/
    ├── 42f7346d-c6d9-4980-9f98-a5879f9c5cb8-m0.avro  (7.3 KB)
    ├── 42f7346d-c6d9-4980-9f98-a5879f9c5cb8-m1.avro  (7.3

### 데이터 파일 vs Delete File 비교

`.files` 메타데이터 테이블로 **현재 활성 상태의 파일**을 비교합니다.  
COW는 데이터 파일만, MOR는 데이터 파일 + Delete File이 혼재합니다.

In [9]:
# COW — 활성 파일 통계
cow_files = spark.sql(f"""
    SELECT content,
           COUNT(*) AS file_count,
           SUM(record_count) AS total_records,
           SUM(file_size_in_bytes) AS total_bytes
    FROM {COW_TABLE}.files
    GROUP BY content
    ORDER BY content
""").collect()

print("COW 활성 파일:")
for row in cow_files:
    content_type = "DATA" if row["content"] == 0 else "DELETES"
    print(f"  {content_type}: {row['file_count']}개 파일, "
          f"{row['total_records']}행, "
          f"{row['total_bytes'] / 1024:.1f} KB")

# MOR — 활성 파일 통계
mor_files = spark.sql(f"""
    SELECT content,
           COUNT(*) AS file_count,
           SUM(record_count) AS total_records,
           SUM(file_size_in_bytes) AS total_bytes
    FROM {MOR_TABLE}.files
    GROUP BY content
    ORDER BY content
""").collect()

print("\nMOR 활성 파일:")
for row in mor_files:
    content_type = "DATA" if row["content"] == 0 else "DELETES"
    print(f"  {content_type}: {row['file_count']}개 파일, "
          f"{row['total_records']}행, "
          f"{row['total_bytes'] / 1024:.1f} KB")

# 비교 요약
cow_data_count = sum(r["file_count"] for r in cow_files if r["content"] == 0)
cow_delete_count = sum(r["file_count"] for r in cow_files if r["content"] == 1)
mor_data_count = sum(r["file_count"] for r in mor_files if r["content"] == 0)
mor_delete_count = sum(r["file_count"] for r in mor_files if r["content"] == 1)

print(f"\n{'─' * 50}")
print(f"{'':15s} {'COW':>10s} {'MOR':>10s}")
print(f"{'─' * 50}")
print(f"{'Data 파일':15s} {cow_data_count:>10d} {mor_data_count:>10d}")
print(f"{'Delete 파일':15s} {cow_delete_count:>10d} {mor_delete_count:>10d}")
print(f"{'총 파일':15s} {cow_data_count + cow_delete_count:>10d} {mor_data_count + mor_delete_count:>10d}")
print(f"{'─' * 50}")

COW 활성 파일:
  DATA: 3개 파일, 1800행, 21.5 KB

MOR 활성 파일:
  DATA: 6개 파일, 2600행, 34.5 KB
  DELETES: 6개 파일, 800행, 11.1 KB

──────────────────────────────────────────────────
                       COW        MOR
──────────────────────────────────────────────────
Data 파일                  3          6
Delete 파일                0          6
총 파일                     3         12
──────────────────────────────────────────────────


### 메타데이터 계층 비교

`metadata.json → Manifest List → Manifest File → Data File` 체인을 통해  
COW와 MOR이 **내부 메타데이터를 어떻게 다르게 구성하는지** 확인합니다.

핵심 차이:
- **COW**: DATA Manifest만 존재 (Delete File이 없으므로)
- **MOR**: DATA Manifest + **DELETES Manifest**가 별도로 존재

In [10]:
print("=" * 60)
print("COW 메타데이터 계층:")
print("=" * 60)
show_metadata_hierarchy(COW_PATH)

print("\n" + "=" * 60)
print("MOR 메타데이터 계층:")
print("=" * 60)
show_metadata_hierarchy(MOR_PATH)

COW 메타데이터 계층:
v4.metadata.json  (operation: overwrite)
│
└─▶ snap-4813404451811166586-1-b8fd46a8-e2ea-4eb6-89a4-fb92b5c91488.avro  [Manifest List]
    ├─▶ b8fd46a8-e2ea-4eb6-89a4-fb92b5c91488-m1.avro  [Manifest — DATA: 3 ADDED]
        │   ├── data/warehouse/lab/cmp_cow_orders/data/order_date_month=2024-01/00000-40-e3e46849-f884-450a-9d73-1f9ff95f495b-00001.parquet  (540행, ADDED)
        │   ├── data/warehouse/lab/cmp_cow_orders/data/order_date_month=2024-02/00000-40-e3e46849-f884-450a-9d73-1f9ff95f495b-00002.parquet  (576행, ADDED)
        │   └── data/warehouse/lab/cmp_cow_orders/data/order_date_month=2024-03/00000-40-e3e46849-f884-450a-9d73-1f9ff95f495b-00003.parquet  (684행, ADDED)
    └─▶ b8fd46a8-e2ea-4eb6-89a4-fb92b5c91488-m0.avro  [Manifest — DATA: 3 DELETED]
            ├── data/warehouse/lab/cmp_cow_orders/data/order_date_month=2024-01/00000-35-9ba3c68f-d801-4864-afc5-91950d1755d0-00001.parquet  (610행, DELETED)
            ├── data/warehouse/lab/cmp_cow_orders/data/order_date_m

### 메타데이터 구조 차이 정리

```
COW의 Manifest List:                   MOR의 Manifest List:
├─▶ Manifest (DATA)  ← DELETE용       ├─▶ Manifest (DATA)      ← INSERT 원본
│   └── 새 파일 (ADDED)               │   └── 원본 파일 (EXISTING)
├─▶ Manifest (DATA)  ← UPDATE용       ├─▶ Manifest (DATA)      ← UPDATE 새 데이터
│   └── 새 파일 (ADDED)               │   └── 새 파일 (ADDED)
└─▶ Manifest (DATA)  ← INSERT 원본    ├─▶ Manifest (DELETES)   ← UPDATE용 Delete File
    └── 원본 파일 (EXISTING/DELETED)   │   └── delete-*.parquet (ADDED)
                                       └─▶ Manifest (DELETES)   ← DELETE용 Delete File
                                           └── delete-*.parquet (ADDED)
```

| 특성 | COW | MOR |
|------|-----|-----|
| Manifest 종류 | DATA만 | DATA + DELETES |
| 기존 파일 상태 | UPDATE/DELETE 시 DELETED로 표시 | EXISTING 유지 (삭제되지 않음) |
| 새 파일 | 전체 데이터 재작성 파일 | 변경분 데이터 파일 + Delete File |
| Manifest 크기 | 재작성된 파일 정보 포함 | 원본 유지 + 작은 추가 파일 정보 |

---
## 종합 비교표

In [11]:
comparison = pd.DataFrame({
    '항목': [
        'INSERT 시간 (초)',
        'UPDATE 시간 (초)',
        'DELETE 시간 (초)',
        'SELECT 시간 (초)',
        '활성 Data 파일 수',
        '활성 Delete 파일 수',
        '총 활성 파일 수',
        '디스크 총 크기 (KB)',
        '최종 레코드 수',
    ],
    'COW': [
        f"{cow_insert_time:.3f}",
        f"{cow_update_time:.3f}",
        f"{cow_delete_time:.3f}",
        f"{cow_read_time:.3f}",
        cow_data_count,
        cow_delete_count,
        cow_data_count + cow_delete_count,
        f"{cow_total / 1024:.1f}",
        cow_count,
    ],
    'MOR': [
        f"{mor_insert_time:.3f}",
        f"{mor_update_time:.3f}",
        f"{mor_delete_time:.3f}",
        f"{mor_read_time:.3f}",
        mor_data_count,
        mor_delete_count,
        mor_data_count + mor_delete_count,
        f"{mor_total / 1024:.1f}",
        mor_count,
    ],
})

print("=" * 65)
print("COW vs MOR 종합 비교 (2,000건, UPDATE 600건, DELETE 200건)")
print("=" * 65)
print(comparison.to_string(index=False))

COW vs MOR 종합 비교 (2,000건, UPDATE 600건, DELETE 200건)
            항목   COW   MOR
 INSERT 시간 (초) 0.494 0.312
 UPDATE 시간 (초) 0.537 0.634
 DELETE 시간 (초) 0.531 0.352
 SELECT 시간 (초) 0.062 0.136
  활성 Data 파일 수     3     6
활성 Delete 파일 수     0     6
     총 활성 파일 수     3    12
 디스크 총 크기 (KB) 130.0 100.2
      최종 레코드 수  1800  1800


---
## 비교 결과 분석

### 1. INSERT — 차이 없음
INSERT는 새 데이터 파일을 생성하는 동일한 동작입니다.  
COW/MOR 전략은 **기존 데이터를 수정할 때만** 개입합니다.

### 2. UPDATE/DELETE — 쓰기 성능에서 MOR 우위
| 관점 | COW | MOR |
|------|-----|-----|
| **쓰기 I/O** | 변경 안 된 행까지 재작성 (쓰기 증폭) | 변경분만 기록 (최소 I/O) |
| **파일 생성** | 기존 파일 크기와 비슷한 새 파일 | 작은 Delete File + 변경분 데이터 |
| **메타데이터** | 기존 파일 DELETED → 새 파일 ADDED | 기존 파일 유지 + Delete File ADDED |

**핵심 인사이트**: 파티션 내 일부 행만 변경해도 COW는 전체 파일을 재작성합니다.  
데이터 파일이 클수록 (수백 MB ~ 수 GB) 이 **쓰기 증폭(write amplification)**의 비용이 커집니다.

### 3. SELECT — 읽기 성능에서 COW 우위
COW 테이블은 항상 최신 상태의 데이터 파일만 존재하므로, 추가 병합 없이 읽을 수 있습니다.  
MOR 테이블은 Delete File을 데이터 파일과 병합해야 하므로, Delete File이 누적될수록 읽기 비용이 증가합니다.

### 4. 파일 구조 — 근본적 차이
```
COW:  데이터 변경 → 파일 재작성 → 항상 "깨끗한" 데이터 파일만 존재
MOR:  데이터 변경 → Delete File 추가 → 시간이 지나면 Delete File 누적 → 컴팩션 필요
```

> **트레이드오프 요약**: COW는 **쓰기 시 비용을 지불**하고, MOR는 **읽기 시 비용을 지불**합니다.  
> MOR의 읽기 비용은 **컴팩션으로 관리**할 수 있으므로, 변경이 빈번한 워크로드에서는 MOR이 유리합니다.

---
## 선택 기준 가이드

| 워크로드 | 추천 | 이유 |
|---------|------|------|
| 읽기 중심, 수정 적음 | **COW** | 읽기 최적화, Delete File 없음 |
| 쓰기 중심, 수정 빈번 | **MOR** | 쓰기 빠름, 정기 컴팩션으로 읽기 비용 관리 |
| 대용량 파티션 + 소량 수정 | **MOR** | COW의 쓰기 증폭이 극대화되는 시나리오 |
| 실시간 분석 (짧은 지연 허용 불가) | **COW** | 읽기 시 병합 비용 없음 |
| 혼합 워크로드 | **작업별 혼합 설정** | DELETE/UPDATE/MERGE를 개별 설정 가능 |

### 실무 팁

- Iceberg에서는 **테이블 속성을 통해 작업별로 다른 전략을 설정**할 수 있습니다
  ```sql
  -- DELETE는 MOR, UPDATE는 COW로 혼합 설정
  ALTER TABLE my_table SET TBLPROPERTIES (
      'write.delete.mode'='merge-on-read',
      'write.update.mode'='copy-on-write'
  )
  ```
- MOR 테이블은 Delete File이 누적되므로, **정기적인 컴팩션(compaction)**이 필요합니다
- 컴팩션은 Delete File을 데이터 파일에 병합하여 읽기 성능을 회복시킵니다
- 프로덕션에서는 **Spark의 `rewrite_data_files`** 프로시저로 컴팩션을 수행합니다

---
## 다음 단계

> MOR 테이블의 읽기 성능을 회복하려면 **정기적인 컴팩션**이 필요합니다.  
> 이 내용은 **4_optimization** 모듈에서 자세히 다룹니다.
>
> 컴팩션을 통해 Delete File을 데이터 파일에 병합하면,  
> MOR의 쓰기 장점을 유지하면서 읽기 성능도 관리할 수 있습니다.

In [12]:
spark.stop()
print("Spark 세션 종료")

Spark 세션 종료
